In [ ]:
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Conv1D, Multiply, Reshape, Add, Conv2D, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import keras
import tensorflow as tf
import os
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Lambda, GlobalAveragePooling2D, Dense, Add, Input, BatchNormalization  # Add BatchNormalization here
from tensorflow.keras.applications import InceptionResNetV2


@keras.saving.register_keras_serializable()
class ECALayer(Layer):
    def __init__(self, k_size=32, **kwargs):
        super(ECALayer, self).__init__(**kwargs)  # Pass **kwargs to parent class
        self.k_size = k_size

    def build(self, input_shape):
        self.global_avg_pool = GlobalAveragePooling2D()
        self.conv1d = Conv1D(1, kernel_size=self.k_size, padding='same', use_bias=False)

    def call(self, inputs):
        # Global Average Pooling
        x = self.global_avg_pool(inputs)
        x = tf.expand_dims(x, axis=-1)  # Expand dims for Conv1D
        x = self.conv1d(x)  # Apply Conv1D for channel attention
        x = tf.keras.activations.sigmoid(x)
        x = tf.squeeze(x, axis=-1)  # Squeeze back the extra dimension
        x = Multiply()([inputs, x])  # Multiply attention with the input
        return x

# HFE Block 
def conv_module(x, k):
    a = Conv2D(k, (1, 1), padding='same')(x)
    a = BatchNormalization()(a)
    a = Activation('swish')(a)

    y = Conv2D(k // 2, (3, 3), padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('swish')(y)
    y = Conv2D(k // 2, (3, 3), padding='same')(y)
    y = BatchNormalization()(y)
    y = Activation('swish')(y)

    z = Conv2D(k, (3, 3), padding='same')(x)
    z = BatchNormalization()(z)
    z = Activation('swish')(z)

    c = tf.keras.layers.Concatenate()([y, z])
    d = Conv2D(k, (3, 3), padding='same')(c)
    d = BatchNormalization()(d)
    d = Activation('swish')(d)

    x = tf.keras.layers.Concatenate()([d, c])
    x = tf.keras.layers.Concatenate()([x, a])
    x = Conv2D(k, (1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    return x

# Data augmentation for the training set
train_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,  # ±15 degrees
    width_shift_range=0.1,  # Horizontal shift
    height_shift_range=0.1,  # Vertical shift
    shear_range=0.1,  # Shear intensity
    zoom_range=[0.8, 1.2],  # Zoom range: 80% to 120%
    fill_mode='nearest',  # Fill mode for empty pixels
    horizontal_flip=True,
)

# No augmentation for validation/test sets, just rescaling
test_data = ImageDataGenerator(rescale=1./255)

# Load train, validation, and test sets using flow_from_directory
train_data = train_data.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=20,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical'
)


test_data = test_data.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=20,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical'
)

# Add Your Developed Inception-ResNet (i.e., Or you can use the pretrained one as follows)
inceptionresnet_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
x = inceptionresnet_base.output
x = conv_module(x, 32)
x = ECALayer()(x)  # Apply ECA block
x = conv_module(x, 64)
x = ECALayer()(x)  # Apply ECA block
# Global pooling
x = GlobalAveragePooling2D()(x)
# Fully connected layer with 128 units
x = Dense(128, activation='relu')(x)
# Output layer for your number of classes (replace 2 with the number of output classes)
output = Dense(2, activation='softmax')(x)

# Define the complete model
model = Model(inputs=densenet_base.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for reducing learning rate and saving the best model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = ModelCheckpoint("model.keras", save_best_only=True, monitor='val_loss')

# Model Training
history = model.fit(
    train_data,
    epochs=20,
    validation_data=test_data,
    callbacks=[reduce_lr, model_checkpoint]
)

# Saving the model
model.save("model.keras")

# Loading the model (with custom objects)
loaded_model = load_model("model.keras", custom_objects={'ECALayer': ECALayer})
